In [1]:
#
# 앞으로 수업에서 실행이 되었다고 가정하는 코드
#

# 그래프 시각화 시, jupyter에서 바로 표현되게 만드는 magic command
%matplotlib inline

# jupyter의 code cell에서 print() 함수 호출 없이, 복수의 변수명만 입력해도 바로 출력창에 출력값들이 나오도록 해주는 설정입니다.
# 구글에 "multiple output in jupyter withtout print()" 로 검색하면 나오는 코드 중에 하나이니, 딱히 외우거나 하시지 않아도 됩니다.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)    # DataFrame 숫자데이터가 소수점 셋째자리까지 나오도록 설정
pd.set_option('max_columns', None)  # DataFrame에서 모든 컬럼을 다 볼 수 있게 설정

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_column', 1000)
pd.options.display.float_format = '{:.2f}'.format

import re
import requests
import bs4
import pandas as pd
import numpy as np
from datetime import datetime
import datetime
import random
from tqdm.autonotebook import tqdm
import string
import re
import seaborn as sns
import matplotlib.pyplot as plt
import os
plt.rcParams['font.family'] = 'AppleGothic' # 폰트 변경
plt.rcParams['axes.unicode_minus'] = False # 축 값 마이너스 깨짐 해결

In [2]:
filepath = 'F:/mycoding/패캠/데이터분석가 취업/파이널프로젝트/★KDT 참여기업_29cm/data/commerce/'

In [3]:
sampled_events = pd.read_csv(filepath+'sampled_events.csv')
sampled_products = pd.read_csv(filepath+'sampled_products.csv')
sampled_users = pd.read_csv(filepath+'sampled_users.csv')

In [4]:
mergedevents=pd.merge(sampled_events,sampled_products, on= 'item_no', how = 'left')

In [5]:
mergedevents= pd.merge(mergedevents, sampled_users, on = 'user_no', how = 'left')

In [6]:
mergedevents['event_timestamp'] = pd.to_datetime(mergedevents['event_timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')

In [7]:
mergedevents=mergedevents.astype({'category1_code':'object'})
mergedevents=mergedevents.astype({'category2_code':'object'})
mergedevents=mergedevents.astype({'category3_code':'object'})
mergedevents=mergedevents.astype({'brand_no':'object'})
mergedevents['birth_date'] = pd.to_datetime(mergedevents['birth_date'], format='%Y-%m-%d', errors='coerce')

# 데이터 결측값 탐구

In [8]:
null_item_list = mergedevents.loc[mergedevents['brand_name'].isnull(),['brand_name','item_name','category1_name','category2_name','category3_name','price']]
mergedevents.loc[null_item_list.query('category2_name == "주얼리"')['item_name'].index,['brand_name']]='nulll'
mergedevents.loc[null_item_list.query('item_name == "[프리미엄침구] 시원한 시어서커/인견 여름이불 모음"')['item_name'].index,['brand_name']]='오르다'
mergedevents.loc[null_item_list.query('item_name == "EMF 전자파차단 침대 거실 프리미엄 전기요"')['item_name'].index,['brand_name']]='한스갤러리'
mergedevents.loc[null_item_list.query('item_name == "그린티 모이스트 파하 거즈 필링(30매)"')['item_name'].index,['brand_name']]='네오젠'
## 주얼리의 경우 브랜드 null 로 확인 브랜드 네임 null_로 변경

In [9]:
# _ 기준으로 자동화 해보자 인생 참 트로트다
# _ 있는 항목들만 우선 추림. 

brandname_1=[]
brandname_2=[]
brandname_total=[]
for i in mergedevents[mergedevents['brand_name'].str.rfind('_')!=-1]['brand_name'].unique():
    idx = i.rfind('_')
    brandname_1.append(i[:idx])
    brandname_2.append(i[idx+1:])
    brandname_total.append(i)

In [10]:
branddf = pd.DataFrame(columns=['brandname_1','brandname_2','brandname_total'])

In [11]:
branddf['brandname_1']= brandname_1
branddf['brandname_2']= brandname_2
branddf['brandname_total']= brandname_total
branddf['no']=1

In [12]:
branddf.head()

,brandname_1,brandname_2,brandname_total,no
0,TAW&TOE,이누,TAW&TOE_이누,1
1,TOMMY HILFIGER,주식회사 이랜드리테일,TOMMY HILFIGER_주식회사 이랜드리테일,1
2,apple,kt,apple_kt,1
3,drmartens,쿠폰 적용,drmartens_쿠폰 적용,1
4,loiter,loiter,loiter_loiter,1


In [13]:
not_brandlist = list(set(mergedevents.brand_name.unique()) - set(branddf.brandname_total.unique()))

In [14]:
## mergedevents 에서 brand_name_df 만
brand_name_df = mergedevents[['brand_name','brand_no']].drop_duplicates().reset_index(drop=True)
brand_name_df['new_brand_name']=''

In [15]:
## 확실한 분류를 먼저 보자.... 
test_under_1 = [
                '_이랜드리테일','_바이스트','_트렌드메카',
                '_마이트레이딩홀딩스','_브랜드 잇','_원츠',
                '_모어디','_에스엠케이', '_주식회사', '_올아이원트',
                '_옐로우바나나', '_ 다올','_뉴욕트레이딩',
                '_올포원컴퍼니','_트렉시','_더팩토리', '무료',
                '_ 주식회사', '_(주)', '_3만', '_1', '_5만',
                '(주)','_조아무역', '_ACC','_비컴퍼니','_비비엘유이',
                '_풋스케이프','_자로라코리아','_사이브로(42BRO)',
                '_이누','_쿠폰 적용','_kt','_착불'
               ]

In [16]:
# 2 braddf 에서 for 문으로 돌림 
for i in test_under_1:
    branddf_ = branddf[branddf['brandname_total'].str.contains(i)]['brandname_total'].unique()
   
    for j in branddf_ : 
        # mergedevents.query(f'brand_name=="{j}"')['new_brand_name'] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]
        brand_name_df.loc[brand_name_df['brand_name']==j, ['new_brand_name']] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]

In [17]:
# # 2 braddf 에서 for 문으로 돌림 
# for i in test_under_1:
#     branddf_ = branddf[branddf['brandname_total'].str.contains(i)]['brandname_total'].unique()
   
#     for j in branddf_ : 
#         # mergedevents.query(f'brand_name=="{j}"')['new_brand_name'] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]
#         mergedevents.loc[mergedevents['brand_name']==j, ['new_brand_name']] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]

In [18]:
## 확실한 것 들어갔는지 확인
## 안들어간것 중에 _ 확인
regex = '_'+r'[가-힣]+'
regex_no = '_'+r'[0-9]+'
diff_data_check = list(set(branddf['brandname_1']) - set(brand_name_df[brand_name_df['new_brand_name']!=''].brand_name))
for i in  diff_data_check: 
    ## _한글 포함했는지 확인?? ## 
    for regex_idx in [regex,regex_no]:
        powerfulremove = re.findall(regex_idx,i)
        if len(powerfulremove) > 0:
            branddf_ = branddf[branddf['brandname_total'].str.contains(powerfulremove[0])]['brandname_total'].unique()
            for j in branddf_ : 
                # mergedevents.query(f'brand_name=="{j}"')['new_brand_name'] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]
                brand_name_df.loc[brand_name_df['brand_name']==j, ['new_brand_name']] = branddf.loc[branddf['brandname_total']==f'{j}'].reset_index()['brandname_1'][0]
## 다시 제거 

In [19]:
### _ 두개로 인해서 잘리지 않은 부분 재정리

for i in ['_ACC','_스피드썬','_한사랑','_다솜','_무료','_SDF','_B','_회산디지털','_NCpicks','_코멧','_평강','코인비엠에스','프로젝트이든','그룹리츠','케이디엘','ACC','_디뷰','_KT','_(Y)','_조건','_디뷰','_현성','_보보','_다원','_최강쇼핑']:
    a = brand_name_df[brand_name_df.new_brand_name.str.contains(i)].index
    if len(a) > 0 :
        print(a)
        for j in a :
            brand_name_df.iloc[j,-1]=brand_name_df.loc[brand_name_df[brand_name_df.new_brand_name.str.contains(i)].index[0],'new_brand_name'].split('_')[0]
            

Int64Index([217, 1484, 2281, 6502], dtype='int64')
Int64Index([7851], dtype='int64')
Int64Index([5369], dtype='int64')
Int64Index([5064], dtype='int64')
Int64Index([2569], dtype='int64')
Int64Index([1042, 3246], dtype='int64')
Int64Index([4620], dtype='int64')
Int64Index([7328], dtype='int64')
Int64Index([7567], dtype='int64')
Int64Index([5090], dtype='int64')
Int64Index([7355], dtype='int64')
Int64Index([1752, 3497], dtype='int64')
Int64Index([1432, 4182], dtype='int64')
Int64Index([5065], dtype='int64')
Int64Index([2140, 2708], dtype='int64')
Int64Index([2310], dtype='int64')
Int64Index([3253], dtype='int64')
Int64Index([4169], dtype='int64')
Int64Index([3233], dtype='int64')
Int64Index([4677], dtype='int64')
Int64Index([4958, 6290], dtype='int64')


In [20]:
len(brand_name_df[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == '')])

628

In [21]:
(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == '')

0       False
1       False
2       False
3       False
4       False
        ...  
7895    False
7896    False
7897    False
7898    False
7899    False
Length: 7900, dtype: bool

In [22]:
## 좌우 공백 제거
brand_name_df['new_brand_name'] = brand_name_df['new_brand_name'].str.strip()

In [23]:
## 띄어쓰기 제거 함수 안먹음.. 
def text_replace_for_df(x):
    x.replace(" ",'')
    return x
brand_name_df['new_brand_name'] = brand_name_df['new_brand_name'].apply(text_replace_for_df)

In [24]:
## str 로 쓰니까 먹음 
brand_name_df['new_brand_name']= brand_name_df['new_brand_name'].str.replace(" ","")

In [25]:
brand_name_df['new_brand_name'] = brand_name_df['new_brand_name'].str.upper()

In [26]:
# ### brand_name_df 에서 brand_name _한글 아직 포함된경우가 있어서 다시.. 
# def underbar_korean(x):
#     regex = '_'+r'[가-힣]+'
#     underbar_korean_remove = re.findall(regex,x)
#     if len(underbar_korean_remove) > 0 : 
#         return x.strip(underbar_korean_remove[0])
#     else :
#         return ""

In [27]:
### brand_name_df 에서 brand_name _한글 아직 포함된경우가 있어서 다시.. 
def underbar_korean(x):
    p=re.compile('_[가-힣].*')
    underbar_korean_remove = p.findall(x)
    if len(underbar_korean_remove) > 0 : 
        return x.strip(underbar_korean_remove[0])
    else :
        return ""

In [28]:
### brand_name_df 에서 brand_name _한글 아직 포함된경우가 있어서 다시.. 
def underbar_korean_num(x):
    p=re.compile('_[0-9].*')
    underbar_korean_remove = p.findall(x)
    if len(underbar_korean_remove) > 0 : 
        return x.strip(underbar_korean_remove[0])
    else :
        return ""

In [29]:
brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == ''),'new_brand_name'] = brand_name_df.brand_name.apply(underbar_korean)
brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == ''),'new_brand_name'] = brand_name_df.brand_name.apply(underbar_korean_num)

In [30]:
### brand_name_df.brand_name 에서 _ 로 끝나는 것들은 제거 

In [31]:
# def underbar_only(x):
#     p=re.compile('_$')
#     underbar_only_ = p.findall(x)
#     if len(underbar_only_) > 0 : 
#         return x.replace('_','')

In [32]:
# def underbar_b(x):
#     p=re.compile('_B')
#     underbar_only_ = p.findall(x)
#     if len(underbar_only_) > 0 : 
#         return x.replace('_','')

In [33]:
# def underbar_sdf(x):
#     p=re.compile('_SDF')
#     underbar_only_ = p.findall(x)
#     if len(underbar_only_) > 0 : 
#         return x.replace('_','')

In [34]:
def underbar_all(x):
    for i in ['_$','_B', '_SDF', '_LS.*', '_[(].*','_INC']:
        p=re.compile(i)
        underbar_only_ = p.findall(x)
        if len(underbar_only_) > 0 : 
            return x.replace('_','')

In [35]:
# brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == ''),'new_brand_name'] = brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name == ''),'brand_name'].apply(underbar_only)

In [36]:
# brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'new_brand_name'] = brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'brand_name'].apply(underbar_only)
# brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'new_brand_name'] = brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'brand_name'].apply(underbar_b)
# brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'new_brand_name'] = brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'brand_name'].apply(underbar_sdf)
brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'new_brand_name'] = brand_name_df.loc[(brand_name_df.brand_name.str.contains('_')) & (brand_name_df.new_brand_name.isnull()),'brand_name'].apply(underbar_all)

In [37]:
brand_name_df.loc[brand_name_df.new_brand_name.isnull(),'new_brand_name']=brand_name_df.loc[brand_name_df.new_brand_name.isnull(),'brand_name']

In [38]:
## _ 있는데 new_brand_name 에 안 옮겨진 항목이 있어서 해당건만 다시 작업
branddf__ = branddf
branddf__.rename(columns={'brandname_total':'brand_name'},inplace=True)
brand_name_df_ = pd.merge(brand_name_df[brand_name_df['new_brand_name']==''],branddf__, on='brand_name', how = 'left')

In [39]:
brand_name_df_['brand_name_']=brand_name_df_['brand_name'].str.upper().str.replace(" ","")

In [40]:
brand_name_df_['new_brand_name'] = brand_name_df_.brand_name_.apply(underbar_korean)

In [41]:
brand_name_df_.loc[brand_name_df_.new_brand_name == '','new_brand_name']=brand_name_df_.loc[brand_name_df_.new_brand_name == '','brand_name']

In [42]:
brand_name_df_.head()

,brand_name,brand_no,new_brand_name,brandname_1,brandname_2,no,brand_name_
0,TOPTEN,13901.00,TOPTEN,NaN,NaN,nan,TOPTEN
1,Wave union,7604.00,Wave union,NaN,NaN,nan,WAVEUNION
2,MONGDOL,11472.00,MONGDOL,NaN,NaN,nan,MONGDOL
3,yeomim,3135.00,yeomim,NaN,NaN,nan,YEOMIM
4,Rockfish Weatherwear,7036.00,Rockfish Weatherwear,NaN,NaN,nan,ROCKFISHWEATHERWEAR


In [43]:
brand_name_df__ = pd.merge(brand_name_df,brand_name_df_[['brand_no','new_brand_name']],on='brand_no',how='left')

In [44]:
brand_name_df_[['brand_no','new_brand_name']]

,brand_no,new_brand_name
0,13901.00,TOPTEN
1,7604.00,Wave union
2,11472.00,MONGDOL
3,3135.00,yeomim
4,7036.00,Rockfish Weatherwear
...,...,...
6885,15042.00,소다스트림
6886,14066.00,퍼펙토
6887,15257.00,어나더스케줄
6888,15265.00,보틀웍스


In [45]:
brand_name_df__[brand_name_df__.new_brand_name_x==""].brand_name.str.contains('_',na=False)

0       False
1       False
2       False
3       False
4       False
        ...  
7898    False
7900    False
7901    False
7902    False
7904    False
Name: brand_name, Length: 6896, dtype: bool

In [46]:
brand_name_df__.loc[brand_name_df__.new_brand_name_x == '','new_brand_name_x'] = brand_name_df__.loc[brand_name_df__.new_brand_name_x == '','new_brand_name_y']

In [47]:
underbar_all('DENOVO__')

'DENOVO'

In [48]:
brand_name_df__.loc[brand_name_df__.new_brand_name_x.str.contains('_'),'new_brand_name_x'] = brand_name_df__[brand_name_df__.new_brand_name_x.str.contains('_')].new_brand_name_x.apply(underbar_all)

In [49]:
brand_name_df__.loc[brand_name_df__.new_brand_name_x.isna(),'new_brand_name_x'] = brand_name_df__.loc[brand_name_df__.new_brand_name_x.isna(),'brand_name']

In [53]:
brand_name_df__.loc[brand_name_df__.brand_no.isnull(),'new_brand_name_x'] = brand_name_df__.loc[brand_name_df__.brand_no.isnull(),'brand_name']

In [55]:
brand_name_df__.loc[brand_name_df__.brand_no.isnull()] 

,brand_name,brand_no,new_brand_name_x,new_brand_name_y
4468,네오젠,NaN,네오젠,네오젠
4469,네오젠,NaN,네오젠,오르다
4470,네오젠,NaN,네오젠,한스갤러리
6913,오르다,NaN,오르다,네오젠
6914,오르다,NaN,오르다,오르다
6915,오르다,NaN,오르다,한스갤러리
7574,한스갤러리,NaN,한스갤러리,네오젠
7575,한스갤러리,NaN,한스갤러리,오르다
7576,한스갤러리,NaN,한스갤러리,한스갤러리


In [56]:
brand_name_df__= brand_name_df__[['brand_no','new_brand_name_x']]

In [58]:
brand_name_df__.rename(columns={'new_brand_name_x':'brand_name'})

,brand_no,brand_name
0,13901.00,TOPTEN
1,7604.00,Wave union
2,11472.00,MONGDOL
3,3135.00,yeomim
4,7036.00,Rockfish Weatherwear
...,...,...
7901,15257.00,어나더스케줄
7902,15265.00,보틀웍스
7903,15447.00,보틀웍스
7904,15526.00,LAZYDONUT


In [50]:
brand_name_df__.to_excel('./brand_name_df__.xlsx')